# Pipeline for cleaning and transformations

In [1]:
import pandas as pd

Variables:

* Income
* Age
* Sex
* Race
* Insurance
* Education

This test is going to be for a single state (Alaska AK) and then modularized to be used to all states

**Important note:**  summary level 50 should be used to have county fips

## Income variables

All refer to HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) for Households

* b19001_002: <10,000
* b19001_003: 10,000 ~ 14,999
* b19001_004: 15,000 ~ 19,999
* b19001_005: 20,000 ~ 24,999
* b19001_006: 25,000 ~ 29,999
* b19001_007: 30,000 ~ 34,999
* b19001_008: 35,000 ~ 39,999
* b19001_009: 40,000 ~ 44,999
* b19001_010: 45,000 ~ 49,999
* b19001_011: 50,000 ~ 59,999
* b19001_012: 60,000 ~ 74,999
* b19001_013: 75,000 ~ 99,999
* b19001_014: 100,000 ~ 124,999
* b19001_015: 125,000 ~ 149,999
* b19001_016: 150,000 ~ 199,999
* b19001_017: 200,000+

This are the variables that we're going to use for our analysis. Other features are not going to be eliminated

In [3]:
income_raw = pd.read_csv('AK_income.csv')
income_raw.head(1)

,SummaryLevel,State,StateFIPS,CountyFIPS,PlaceFIPS,CBSACode,CongressDistrict,GEOID,AreaName,B19001_001,...,B19325_093,B19325_094,B19325_095,B19326_001,B19326_002,B19326_003,B19326_004,B19326_005,B19326_006,B19326_007
0,40,AK,2,NaN,NaN,NaN,NaN,04000US02,Alaska,250235,...,2953,3527,2450,31944,40197,61240,19058,25706,47792,13310


## Age & Sex variables

This features refer to SEX BY AGE for Total Population (Male)

* b01001_003: <5
* b01001_004: 5 ~ 9
* b01001_005: 10 ~ 14
* b01001_006: 15 ~ 17
* b01001_007: 18 ~ 19
* b01001_008: 20
* b01001_009: 21
* b01001_010: 22 ~ 24
* b01001_011: 25 ~ 29
* b01001_012: 30 ~ 34
* b01001_013: 35 ~ 39
* b01001_014: 40 ~ 44
* b01001_015: 45 ~ 49
* b01001_016: 50 ~ 54
* b01001_017: 55 ~ 59
* b01001_018: 60 ~ 61
* b01001_019: 62 ~ 64
* b01001_020: 65 ~ 66
* b01001_021: 67 ~ 69
* b01001_022: 70 ~ 74
* b01001_023: 75 ~ 79
* b01001_024: 80 ~ 84
* b01001_025: 85+

This features refer to SEX BY AGE for Total Population (Female)

* b01001_027: <5
* b01001_028: 5 ~ 9
* b01001_029: 10 ~ 14
* b01001_030: 15 ~ 17
* b01001_031: 18 ~ 19
* b01001_032: 20
* b01001_033: 21
* b01001_034: 22 ~ 24
* b01001_035: 25 ~ 29
* b01001_036: 30 ~ 34
* b01001_037: 35 ~ 39
* b01001_038: 40 ~ 44
* b01001_039: 45 ~ 49
* b01001_040: 50 ~ 54
* b01001_041: 55 ~ 59
* b01001_042: 60 ~ 61
* b01001_043: 62 ~ 64
* b01001_044: 65 ~ 66
* b01001_045: 67 ~ 69
* b01001_046: 70 ~ 74
* b01001_047: 75 ~ 79
* b01001_048: 80 ~ 84
* b01001_049: 85+

In [4]:
age_sex_raw = pd.read_csv('AK_age_sex.csv')
age_sex_raw.head(1)

,SummaryLevel,State,StateFIPS,CountyFIPS,PlaceFIPS,CBSACode,CongressDistrict,GEOID,AreaName,B01001_001,...,B01002G_001,B01002G_002,B01002G_003,B01002H_001,B01002H_002,B01002H_003,B01002I_001,B01002I_002,B01002I_003,B01003_001
0,40,AK,2,NaN,NaN,NaN,NaN,04000US02,Alaska,736855,...,19.7,19.3,20.2,38.2,37.9,38.6,24.9,24.5,25.6,736855


## Race variable

This features refer to RACE for Total Population

* b02001_002: White alone
* b02001_003: Black or African American alone
* b02001_004: American Indian and Alaska Native alone
* b02001_005: Asian alone
* b02001_006: Native Hawaiian and Other Pacific Islander alone
* b02001_007: Some other race alone
* b02001_008: Two or more races

In [5]:
race_raw = pd.read_csv('AK_race.csv')
race_raw.head(1)

,SummaryLevel,State,StateFIPS,CountyFIPS,PlaceFIPS,CBSACode,CongressDistrict,GEOID,AreaName,B02001_001,...,C02003_010,C02003_011,C02003_012,C02003_013,C02003_014,C02003_015,C02003_016,C02003_017,C02003_018,C02003_019
0,40,AK,2,NaN,NaN,NaN,NaN,04000US02,Alaska,736855,...,3056,59284,56929,7319,32356,7883,1924,7447,4767,644


## Health insurance variable

* b27001_001: HEALTH INSURANCE COVERAGE
* b27002_001: PRIVATE HEALTH INSURANCE
* b27003_001: PUBLIC HEALTH INSURANCE
* c27004_001: EMPLOYER-BASED HEALTH INSURANCE 
* c27005_001: DIRECT-PURCHASE HEALTH INSURANCE
* c27006_001: MEDICARE COVERAGE
* c27007_001: MEDICAID/MEANS-TESTED PUBLIC COVERAGE
* c27008_001: TRICARE/MILITARY HEALTH COVERAGE
* c27009_001: VA HEALTH CARE




In [ ]:
insurance_raw = pd.read_csv('AK_insurance.csv')
insurance_raw.head(1)

In [ ]:
## Education variable

